## Consideraciones del dataset

La encuesta que se realizó para la obtención del dataset se encuentra dividida en 2 bloques:

- Bloque hogar: 
Que releva datos de condiciones habitacionales, características demográficas, educativas de todas las personas de la vivienda

- Bloque individual:
Datos laborales e información sobre factores de riesgo de 1 solo componente del hogar (seleccionado por tabla de kish)
Este bloque se subdivide en 3 partes:
    1. Información con autoreporte (trabajo, peso, tabaco, actividad física, etc)
    2. Mediciones antropométricas (presion, altura, talla) realizadas por personal
    3. Mediciones bioquímicas (glucemia, colesterol)
    
### Dominios de estimación

El 100% de las viviendas forman parte del paso 1
El paso 2 y como consecuencia de la muestra de viviendas en un 25%, las estimaciones responden al total del país.
Para el paso 3, al ser un recorte geográfico de la muestra del paso 2, se define un único dominio compuesto por el conjunto de algomeradod o localidades de 150.000 habitantes y más

### Limitaciones del análisis

En base a los recortes de la encuesta, solo se analizarán el bloque hogar y el bloque individual hasta el punto 1. Las mediciones antropométricas y bioquímicas no formarán parte de este análisis ya que se considera como  válido lo reportado por los individuos y no es el objetivo de este análisis conocer si las respuestas de los individuos respecto a su características de salud son acertadas o no.
Solamente se analizarán los resultados de la encuesta realizada en el año 2018.

-------------------------------------------------------
    

### En esta notebook se realiza el análisis por cada columna para seleccionar cuáles se utilizarán, cuales se desecharán y cuales se transformarán o modificarán (por ejemplo mediante un merge) para poder realizar un mejor análisis

In [1]:
#importanción de librerías a utilizar

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import sklearn as skl
from sklearn.preprocessing import MinMaxScaler
from colorama import init, Fore, Style

   
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import joblib

In [2]:
# IMPORTACIÓN DEL DATASET (resultado de la encuesta del año 2018)

friesgo_2018= pd.read_csv('friesgo2018.txt', delimiter = '|')



encoder = OneHotEncoder(sparse_output=False, drop='first')
#scaler = StandardScaler()


In [3]:
#columnas a utilizar para el análisis actual (se decartan el resto solo a los fines del análisis actual)

dominio_geografico = friesgo_2018.iloc[:, :7]
caracteristicas_vivienda = friesgo_2018.iloc[:, list(range(0,1))+ list(range(7, 18))]
caracteristicas_hogar = friesgo_2018.iloc[:, list(range(0,1))+ list(range(18, 24))]
ingresos_hogar = friesgo_2018.iloc[:, list(range(0,1))+ list(range(24, 30))]

caracteristicas_encuestado = friesgo_2018.iloc[:,list(range(0,1))+ list(range(51,66))]
salud_y_af = friesgo_2018.iloc[:,list(range(0,1))+ list(range(73,98))]                                                     
tabaquismo = friesgo_2018.iloc[:,list(range(0,1))+ list(range(98, 132))]
hipertension = friesgo_2018.iloc[:,list(range(0,1))+ list(range(140,160))]
pesocorporal = friesgo_2018.iloc[:,list(range(0,1))+ list(range(160,168))]
alimentacion = friesgo_2018.iloc[:,list(range(0,1))+ list(range(169,186))]
colesterol = friesgo_2018.iloc[:,list(range(0,1))+ list(range(187,195))]
consumo_alcohol = friesgo_2018.iloc[:,list(range(0,1))+ list(range(195,217))]
diabetes = friesgo_2018.iloc[:,list(range(0,1))+ list(range(218,236))]

#mediciones_antropometricas = friesgo_2018.iloc[:,list(range(0,1))+ list(range(252,273))]
#mediciones_bioquimicas = friesgo_2018.iloc[:,list(range(0,1))+ list(range(274,283))]



## 1) DOMINIO GEOGRÁFICO


In [4]:
dominio_geografico.head(5)

,id,cod_provincia,region,tamanio_aglomerado,aglomerado,localidades_150,submuestra
0,1128639,2,1,1,1,1,1
1,1709939,2,1,1,1,1,0
2,6874130,2,1,1,1,1,0
3,10319375,2,1,1,1,1,1
4,11140857,2,1,1,1,1,0


In [5]:
# ELIMINO COLUMNAS NO USADAS

# se eliminan las columnas cod_provincia ya que se analizarán REGIONES, no provincias en particular.
#se elimina la columna de aglomerado, ya que con región y tamaño de aglomerado se pueden inferir diferencias
# se elimina submuestra porque no se considerarán las respuestas derivadas de esto

dominio_geografico = dominio_geografico.drop(columns = ['cod_provincia', 'aglomerado', 'localidades_150','submuestra'] )

In [6]:
# VALORES NULOS

print(f"Valores nulos: {dominio_geografico.isnull().sum()}\n") 


Valores nulos: id                    0
region                0
tamanio_aglomerado    0
dtype: int64



In [7]:
#encoder y escaler

df =  dominio_geografico

columnas_codificar = ['tamanio_aglomerado']
#columnas_escalar = ['col2', 'col4', 'col6']

columnas_codificadas = encoder.fit_transform(df[columnas_codificar])
df_codif = pd.DataFrame(columnas_codificadas, columns=encoder.get_feature_names_out(columnas_codificar))
df_codif = df_codif.astype(int)  # Convertir a int

#escalar 
#df[columnas_escalar] = scaler.fit_transform(df[columnas_escalar])
                #df_codif = pd.DataFrame(scaler.fit_transform(df_codif), columns=df_codif.columns)

#uno todo

df = df.drop(columnas_codificar, axis=1)
df = pd.concat([df, df_codif], axis=1)

dominio_geografico = df


In [8]:
dominio_geografico.head(5)

,id,region,tamanio_aglomerado_2,tamanio_aglomerado_3,tamanio_aglomerado_4
0,1128639,1,0,0,0
1,1709939,1,0,0,0
2,6874130,1,0,0,0
3,10319375,1,0,0,0
4,11140857,1,0,0,0


In [9]:
#DICCIONARIO POR SI LO NECESITO

#        region_dict = {
#            1: 'Metropolitana',
#            2: 'Pampeana',
#            3: 'Noroeste',
#            4: 'Noreste',
#            5: 'Cuyo',
#            6: 'Patagónica'
#        }
#        dominio_geografico['region'] = dominio_geografico['region'].map(region_dict)


#        tamanio_aglomerado_dict = {
#            1: 'más de 1.500.000 habitantes',
#            2: '500.001 a 1.500.000 habitantes',
#            3: '150.001 a 500.000 habitantes',
#            4: 'Menos de 150.000 habitantes'
#        
#        }
#        dominio_geografico['tamanio_aglomerado'] = dominio_geografico['tamanio_aglomerado'].map(tamanio_aglomerado_dict)

## 2) CARACTERÍSTICAS DE LA VIVIENDA


In [10]:
caracteristicas_vivienda.head(5)



,id,bhcv01,bhcv02,bhcv03,bhcv04,bhcv05,bhcv06,bhcv07,bhcv08,bhcv09,bhcv10,bhcv11
0,1128639,3,2,1,2,1,1,1,1,1,1.0,1.0
1,1709939,1,4,1,2,1,1,1,1,1,1.0,1.0
2,6874130,3,2,1,2,1,1,1,1,1,1.0,1.0
3,10319375,3,1,1,2,1,4,1,1,1,1.0,1.0
4,11140857,3,3,1,2,1,1,1,1,1,1.0,1.0


In [11]:
#ELIMINO COLUMNAS QUE NO USO Y RENOMBRO

#desde la columnas eliminadas corresponden a detalles muy específicos de las viviendas que no se incluirán en el análisis

caracteristicas_vivienda = caracteristicas_vivienda.drop(columns=['bhcv02','bhcv03', 'bhcv04', 'bhcv05', 'bhcv07', 'bhcv10' ])

caracteristicas_vivienda.rename(columns = {
    "bhcv01" : "tipo_vivienda",
    "bhcv06" : "obtiene_gas",
    "bhcv08" : "obtiene_agua",
    "bhcv09" : "tiene_baño",
    "bhcv11" : "desague_baño",
}, inplace = True)
  

In [12]:
#VALORES NULOS      

#print(caracteristicas_vivienda.head(5))
print(f"Valores nulos: {caracteristicas_vivienda.isnull().sum()}\n") 

(caracteristicas_vivienda['tiene_baño']!= 1).sum()

Valores nulos: id                 0
tipo_vivienda      0
obtiene_gas        0
obtiene_agua       0
tiene_baño         0
desague_baño     122
dtype: int64



122

### Se observa que las filas que tienen nulos en desague de baño corresponden a las casas que no tienen baño ya que ambas dan 122. Se combinarán las columnas tiene baño y desague de baño para que haya 1 sola columna con baño y que diga no tiene o que si tiene indique el desague


In [13]:
# ELIMINO COLUMNAS, RENOMBRO Y COMBINO

#combino columnas tiene baño y desague baño para que en una sola columna indique si tiene baño o no y cual es el tipo de desague
caracteristicas_vivienda['baño'] = np.where(caracteristicas_vivienda['tiene_baño'] == 1, caracteristicas_vivienda['desague_baño'], 0)


#Elimino las columnas individuales y solo dejo la combinada
caracteristicas_vivienda = caracteristicas_vivienda.drop(columns=['desague_baño','tiene_baño' ])

### TIPOS DE VARIABLES

In [14]:
#EVALÚO TIPOS DE VARIABLES

# Convertimos los datos de las columna 'BAÑO' a enteros ya que son de tipo float

print(caracteristicas_vivienda.dtypes)

caracteristicas_vivienda['baño'] = caracteristicas_vivienda['baño'].astype(int)
print(caracteristicas_vivienda.dtypes)


#nuevo_Data = pd.DataFrame(columns=caracteristicas_vivienda.columns)

#for index, row in caracteristicas_vivienda.iterrows():
#    if row['baño'] == 0:
#        nuevo_Data = pd.concat([nuevo_Data, pd.DataFrame([row])], ignore_index=True)

        
#nuevo_Data.head(20)

id                 int64
tipo_vivienda      int64
obtiene_gas        int64
obtiene_agua       int64
baño             float64
dtype: object
id               int64
tipo_vivienda    int64
obtiene_gas      int64
obtiene_agua     int64
baño             int32
dtype: object


In [15]:
caracteristicas_vivienda.columns

Index(['id', 'tipo_vivienda', 'obtiene_gas', 'obtiene_agua', 'baño'], dtype='object')

In [16]:
#encoder y escaler

df =  caracteristicas_vivienda

columnas_codificar = ['tipo_vivienda', 'obtiene_gas', 'obtiene_agua', 'baño']
#columnas_escalar = ['col2', 'col4', 'col6']

columnas_codificadas = encoder.fit_transform(df[columnas_codificar])
df_codif = pd.DataFrame(columnas_codificadas, columns=encoder.get_feature_names_out(columnas_codificar))
df_codif = df_codif.astype(int)  # Convertir a int

#escalar 
#df[columnas_escalar] = scaler.fit_transform(df[columnas_escalar])
                

#uno todo

df = df.drop(columnas_codificar, axis=1)
df = pd.concat([df, df_codif], axis=1)

caracteristicas_vivienda = df

In [17]:
caracteristicas_vivienda.head(5)

,id,tipo_vivienda_2,tipo_vivienda_3,tipo_vivienda_4,tipo_vivienda_5,tipo_vivienda_6,tipo_vivienda_7,obtiene_gas_2,obtiene_gas_3,obtiene_gas_4,obtiene_agua_2,obtiene_agua_3,obtiene_agua_4,obtiene_agua_5,baño_1,baño_2,baño_3,baño_4
0,1128639,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,1709939,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,6874130,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,10319375,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0
4,11140857,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


### DICCIONARIO

In [18]:
#tipo_vivienda_dict = {
#    1: 'casa',
#    2: 'casilla',
#    3: 'departamento',
#    4: 'pieza de inquilinato',
#    5: 'pieza en hotel o pension',
#    6: 'local no construido para pension',
#    7: 'otros'
#}

#caracteristicas_vivienda['tipo_vivienda'] = caracteristicas_vivienda['tipo_vivienda'].map(tipo_vivienda_dict)

                                 
#obtiene_gas_dict = {
#    1: 'red',
#    2: 'tubo/garrafa',
#    3: 'kerosen/leña/carbón',
#    4: 'otro'
#}

#caracteristicas_vivienda['obtiene_gas'] = caracteristicas_vivienda['obtiene_gas'].map(obtiene_gas_dict)


#obtiene_agua_dict = {
#    1: 'red publica',
#    2: 'perforación con bomba a motor',
#    3: 'perforación con bomba manual',
#    4: 'aljibe o pozo',
#    5: 'otros',
#}

#caracteristicas_vivienda['obtiene_agua'] = caracteristicas_vivienda['obtiene_agua'].map(obtiene_agua_dict)


#baño_dict = {
#    0: 'sin baño',
#    1: 'cloaca',
#    2: 'camara séptica y pozo ciego',
#    3: 'solo pozo ciego',
#    4: 'excavación en tierra'
#}

#caracteristicas_vivienda['baño'] = caracteristicas_vivienda['baño'].map(baño_dict)

## 3) CARACTERÍSTICAS DEl HOGAR


In [19]:
caracteristicas_hogar.head(5)

,id,bhho01,bhho02,bhho03,cant_componentes,miembros_18,tipo_hogar
0,1128639,1.0,2,1,2,2,2
1,1709939,1.0,4,3,3,3,4
2,6874130,1.0,2,1,2,2,2
3,10319375,1.0,1,1,1,1,1
4,11140857,1.0,3,2,2,2,8


In [20]:
#ELIMINACIÓN DE COLUMNAS

# se elminan columnas: bhho01 = "el baño es exclusivo del hogar"
#bhho02: cuantos ambientes
#bho03 cuartos para dormir
#miembros_18 (ya está detallado la cantidad de miembros del hogar

caracteristicas_hogar = caracteristicas_hogar.drop(columns=['bhho01','bhho02', 'bhho03', 'miembros_18'])



In [21]:
#DICCIONARIO

#tipo_hogar_dict = {
#    1: 'Hogar unipersonal',
#    2: 'Hogar multipersonal conyugal completo sin hijos ni otros miembros',
#    3: 'Hogar multipersonal conyugal completo sin hijos y con otros miembros',
#    4: 'Hogar multipersonal conyugal completo con hijos ni otros miembros',
#    5: 'Hogar multipersonal conyugal completo con hijos y con otros miembros',
#    6: 'Hogar multipersonal conyugal incompleto sin otros miembros',
#    7: 'Hogar multipersonal conyugal incompleto con otros miembros',
#    8: 'Hogar multipersonal no conyugal'
#}

#caracteristicas_hogar['tipo_hogar'] = caracteristicas_hogar['tipo_hogar'].map(tipo_hogar_dict)


In [22]:
#TIPOS DE VARIABLES

caracteristicas_hogar.dtypes

id                  int64
cant_componentes    int64
tipo_hogar          int64
dtype: object

In [23]:
# VALORES NULOS

print(f"Valores nulos: {caracteristicas_hogar.isnull().sum()}\n") 

Valores nulos: id                  0
cant_componentes    0
tipo_hogar          0
dtype: int64



In [24]:
#encoder y escaler

df =  caracteristicas_hogar

columnas_codificar = ['tipo_hogar']
columnas_escalar = ['cant_componentes']

columnas_codificadas = encoder.fit_transform(df[columnas_codificar])
df_codif = pd.DataFrame(columnas_codificadas, columns=encoder.get_feature_names_out(columnas_codificar))
df_codif = df_codif.astype(int)  # Convertir a int

#escalar 
df[columnas_escalar] = scaler.fit_transform(df[columnas_escalar])
                

#uno todo

df = df.drop(columnas_codificar, axis=1)
df = pd.concat([df, df_codif], axis=1)

caracteristicas_hogar = df

## 4) INGRESOS DEl HOGAR

In [25]:
ingresos_hogar.head(5)

,id,bhih01,bhih01_02,rango_ingreso,quintil_uc,imputado,bhih03
0,1128639,50000,NaN,NaN,5,NaN,2
1,1709939,40000,99.0,99.0,5,1.0,2
2,6874130,40000,99.0,12.0,5,1.0,2
3,10319375,80000,NaN,NaN,5,NaN,2
4,11140857,30000,99.0,9.0,5,1.0,2


In [26]:
# ELIMINO Y RENOMBRO COLUMNAS


#solo dejamos columas de ingreso y asignaciones

ingresos_hogar = ingresos_hogar.drop(columns=['bhih01_02','rango_ingreso', 'quintil_uc', 'imputado'])

ingresos_hogar.rename(columns = {
    "bhih01" : "ingresos",
    "bhih03" : "asignaciones",
}, inplace = True)
  
    


In [27]:
#diccionario por si lo necesito


#asignaciones_dict = {
#    1: 'si',
#    2: 'no',
#}

#ingresos_hogar['asignaciones'] = ingresos_hogar['asignaciones'].map(asignaciones_dict)

In [28]:
# tipos de variables

ingresos_hogar.dtypes

id              int64
ingresos        int64
asignaciones    int64
dtype: object

In [29]:
# VALORES NULOS

print(f"Valores nulos: {ingresos_hogar.isnull().sum()}\n") 

Valores nulos: id              0
ingresos        0
asignaciones    0
dtype: int64



In [30]:
#encoder y escaler

df =  ingresos_hogar

columnas_codificar = ['asignaciones']
columnas_escalar = ['ingresos']

columnas_codificadas = encoder.fit_transform(df[columnas_codificar])
df_codif = pd.DataFrame(columnas_codificadas, columns=encoder.get_feature_names_out(columnas_codificar))
df_codif = df_codif.astype(int)  # Convertir a int

#escalar 
df[columnas_escalar] = scaler.fit_transform(df[columnas_escalar])
                

#uno todo

df = df.drop(columnas_codificar, axis=1)
df = pd.concat([df, df_codif], axis=1)

ingresos_hogar = df

## 5) CARACTERÍSTICAS ENCUESTADO

In [31]:
caracteristicas_encuestado.head(5)

,id,bhch02,bhch03,bhch04,rango_edad,bhch05,nivel_instruccion,nivel_instruccion_agrupado,bhch10_01,bhch10_02,bhch10_03,bhch10_04,bhch10_05,bhch10_06,bhch10_99,cobertura_salud
0,1128639,2,1,32,2,6,7,3,NaN,NaN,NaN,NaN,NaN,1.0,NaN,2
1,1709939,2,2,65,5,2,7,3,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1
2,6874130,2,2,33,2,6,7,3,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1
3,10319375,1,2,64,4,3,7,3,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1
4,11140857,9,1,24,1,6,5,3,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1


In [32]:
# RENOMBRO COLUMNAS


caracteristicas_encuestado.rename(columns={
 #     'bhch02':'parentesco_con_jefeHogar',
      'bhch03':'sexo',
      'bhch04':'edad',
      'bhch05':'situacion_conyugal',
      'bhch10_01':'ascociado_Obra_social',
      'bhch10_02':'asociado_prepaga_porObra_social',
      'bhch10_03':'asociado_prepaga',
      'bhch10_04':'asociado_servicio_emergencia',
      'bhch10_05':'asociado_plan_estatal',
      'bhch10_06':'no_asociado_nada',
      'bhch10_99':'obra_social_ns_nc'}, inplace=True) 

In [33]:
## ELIMINO COLUMNAS NO NECESARIAS

# se elimina rango edad ya que ya tenemos la variabel edad iguak que nivel instruccion agrupado ya que se tiene nivel 
#instruccion y cobertura sakuda que solo dice si tiene o no pero se puede inferir de respuestas, parentesco con jefe de hoga no es importante

caracteristicas_encuestado = caracteristicas_encuestado.drop(columns= ['bhch02','rango_edad', 'nivel_instruccion_agrupado','cobertura_salud'] )


 A CONTINUACION DE REALIZA UN PROCESO PARA PODER COMBINAR LAS COLUMNAS:
 
 bhch10_01 bhch10_02 bhch10_03 bhch10_04 bhch10_05 bhch10_06 bhch10_99  
 
 EN UNA SOLA QUE INDIQUE EL  TIPO DE COBRETURA


In [34]:
#creo nueva columna que tendra valores agrupados
#caracteristicas_encuestado['cobertura_salud_nueva'] = 0  #creo nueva columna que tendra valores agrupados

#agrupo columnas correspondientes para imprimir y revisar el proceso
columnas_a_convertir = ['ascociado_Obra_social', 'asociado_prepaga_porObra_social', 'asociado_prepaga' , 'asociado_servicio_emergencia', 'asociado_plan_estatal', 'no_asociado_nada','obra_social_ns_nc']

# lleno los nan con 0 porque la funcion as type da error sino
caracteristicas_encuestado[columnas_a_convertir]= caracteristicas_encuestado[columnas_a_convertir].fillna(0)  # Reemplazar NaN con 0

#paso valores en columnas a entero
caracteristicas_encuestado[columnas_a_convertir] = caracteristicas_encuestado[columnas_a_convertir].astype(int)


#caracteristicas_encuestado['cobertura_salud_nueva'] = np.where(caracteristicas_encuestado['ascociado_Obra_social']== 1 , 1, caracteristicas_encuestado['cobertura_salud_nueva'] )

#caracteristicas_encuestado['cobertura_salud_nueva'] = np.where(caracteristicas_encuestado['asociado_prepaga_porObra_social']== 1 , 2, caracteristicas_encuestado['cobertura_salud_nueva'] )

#caracteristicas_encuestado['cobertura_salud_nueva'] = np.where(caracteristicas_encuestado['asociado_prepaga']== 1 , 3, caracteristicas_encuestado['cobertura_salud_nueva'] )

#caracteristicas_encuestado['cobertura_salud_nueva'] = np.where(caracteristicas_encuestado['asociado_servicio_emergencia']== 1 , 4, caracteristicas_encuestado['cobertura_salud_nueva'] )

#caracteristicas_encuestado['cobertura_salud_nueva'] = np.where(caracteristicas_encuestado['asociado_plan_estatal']== 1 , 5, caracteristicas_encuestado['cobertura_salud_nueva'] )

#caracteristicas_encuestado['cobertura_salud_nueva'] = np.where(caracteristicas_encuestado['no_asociado_nada']== 1 , 6, caracteristicas_encuestado['cobertura_salud_nueva'] )

#caracteristicas_encuestado['cobertura_salud_nueva'] = np.where(caracteristicas_encuestado['ns_nc']== 1 , 7, caracteristicas_encuestado['cobertura_salud_nueva'] )



In [35]:


#Elimino las columnas individuales y solo dejo la combinada
#columnas_a_eliminar = ['ascociado_Obra_social', 'asociado_prepaga_porObra_social', 'asociado_prepaga' , 'asociado_servicio_emergencia', 'asociado_plan_estatal', 'no_asociado_nada','ns_nc']

#caracteristicas_encuestado = caracteristicas_encuestado.drop(columns=columnas_a_eliminar)

#print(caracteristicas_encuestado.head(10))

In [36]:
#DICCIONARIO POR SI LO NECESITO


#situacion_conyugal_dict = {
#    1: 'Unido/a',
#    2: 'Casado/a',
#    3: 'Separado/a',
#    4: 'Divorciado/a',
#    5: 'Viudo/a',
#    6: 'Soltero/a'
#}

#caracteristicas_encuestado['situacion_conyugal'] = caracteristicas_encuestado['situacion_conyugal'].map(situacion_conyugal_dict)




#cobertura_salud_nueva_dict = {
#    1: 'ascociado_Obra_social',
#    2: 'asociado_prepaga_porObra_social',
#    3: 'asociado_prepaga',
#    4: 'asociado_servicio_emergencia',
#    5: 'asociado_plan_estatal',
#    6: 'no_asociado_nada',
#    7: 'ns_nc'
#}

#caracteristicas_encuestado['cobertura_salud_nueva'] = caracteristicas_encuestado['cobertura_salud_nueva'].map(cobertura_salud_nueva_dict)




#nivel_instruccion_dict = {
#    1: 'Sin instruccion',
#    2: 'Primario incompleto',
#    3: 'Primario completo',
#    4: 'Secundario incompleto',
#    5: 'Secundario completo',
#    6: 'Terciario/Universitario incompleto',
#    7: 'Terciario/Universitario completo',
#    8: 'Educacion especial'
#}

#caracteristicas_encuestado['nivel_instruccion'] = caracteristicas_encuestado['nivel_instruccion'].map(nivel_instruccion_dict)

In [37]:
caracteristicas_encuestado.columns

Index(['id', 'sexo', 'edad', 'situacion_conyugal', 'nivel_instruccion',
       'ascociado_Obra_social', 'asociado_prepaga_porObra_social',
       'asociado_prepaga', 'asociado_servicio_emergencia',
       'asociado_plan_estatal', 'no_asociado_nada', 'obra_social_ns_nc'],
      dtype='object')

In [38]:
#encoder y escaler

df =  caracteristicas_encuestado

columnas_codificar = ['situacion_conyugal', 'nivel_instruccion', 'sexo']
columnas_escalar = ['edad' ]

columnas_codificadas = encoder.fit_transform(df[columnas_codificar])
df_codif = pd.DataFrame(columnas_codificadas, columns=encoder.get_feature_names_out(columnas_codificar))
df_codif = df_codif.astype(int)  # Convertir a int

#escalar 
df[columnas_escalar] = scaler.fit_transform(df[columnas_escalar])
                

#uno todo

df = df.drop(columnas_codificar, axis=1)
df = pd.concat([df, df_codif], axis=1)

caracteristicas_encuestado = df

In [39]:
# Guardar el scaler en un archivo
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [40]:
caracteristicas_encuestado.columns

Index(['id', 'edad', 'ascociado_Obra_social',
       'asociado_prepaga_porObra_social', 'asociado_prepaga',
       'asociado_servicio_emergencia', 'asociado_plan_estatal',
       'no_asociado_nada', 'obra_social_ns_nc', 'situacion_conyugal_2',
       'situacion_conyugal_3', 'situacion_conyugal_4', 'situacion_conyugal_5',
       'situacion_conyugal_6', 'nivel_instruccion_2', 'nivel_instruccion_3',
       'nivel_instruccion_4', 'nivel_instruccion_5', 'nivel_instruccion_6',
       'nivel_instruccion_7', 'nivel_instruccion_8', 'sexo_2'],
      dtype='object')

## 6) SALUD Y ACTIVIDAD FÍSICA

In [41]:
salud_y_af.head(5)

,id,bisg01,bisg02,bisg03,bisg04,bisg05,bisg06,biaf01,biaf02_m,biaf02_99,...,biaf07_m,biaf07_99,biaf08,biaf09,biaf10_01,biaf10_02,biaf10_03,biaf10_04,nivel_actividad_fisica,barreras_actividad_fisica
0,1128639,3,1,1,1,1,2,4,500.0,NaN,...,480.0,NaN,NaN,2.0,2.0,2.0,1.0,1.0,1.0,1
1,1709939,1,1,1,1,1,2,3,210.0,NaN,...,600.0,NaN,NaN,2.0,2.0,1.0,1.0,1.0,1.0,1
2,6874130,2,1,1,1,1,1,0,NaN,NaN,...,540.0,NaN,NaN,10.0,1.0,1.0,1.0,1.0,2.0,9
3,10319375,3,1,1,1,2,2,0,NaN,NaN,...,120.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,3.0,1
4,11140857,3,1,1,1,1,2,0,NaN,NaN,...,480.0,NaN,NaN,2.0,1.0,1.0,1.0,2.0,2.0,1


In [42]:
# RENOMBRO COLUMNAS

salud_y_af.rename(columns = {
    #SALUD GENERAL 
    "bisg01" : "salud_actual",
    "bisg02" : "nivel_de_movilidad",
    "bisg03" : "capacidad_cuidado_personal",
    "bisg04" : "capacidad_actividades_cotidianas",
    "bisg05" : "dolor_malestar_general",
    "bisg06" : "ansiedad_depresion",

#ACTIVIDAD FÍSICA  af
    "biaf01" : "af_intensa_veces_por_semana",
    "biaf02_m" : "af_intensa_min_totales_por_semana",
    "biaf02_99" : "af_intensa_min_totales_por_semana_nsnc",
    "biaf03" : "af_moderada_veces_por_semana",
    "biaf04_m" : "af_moderada_min_totales_por_semana",
    "biaf04_99" : "actividad_fisica_moderada_min_totales_por_semana_nsnc",
    "biaf05" : "caminata_veces_por_semana",
    "biaf06_m" : "caminata_min_totales_por_semana",
    "biaf06_99" : "caminata_min_totales_por_semana_nsnc",
    "biaf07_m" : "tiempo_sentado_min_por_dia",
    "biaf07_99" : "tiempo_sentado_min_por_dia_nsnc",
    "biaf08" : "razon_ninguna_actividad_la_ultima_semana",
    "biaf09" : "razon_poca_actividad_la_ultima_semana",
    "biaf10_01" : "af_fue_actividad_domestica",
    "biaf10_02" : "af_fue_actividad_laboral",
    "biaf10_03" : "af_fue_traslado",
    "biaf10_04" : "af_fue_deporte"

    }, inplace = True)

In [43]:
# VALORES NULOS

print(f"Valores nulos: {salud_y_af.isnull().sum()}\n") 

#SE OBSERVA QUE LOS VALORES NAN CORRESPONDEN A COLUMNAS QUE POSIBLEMENTE TENGAN ALGUNAS RESPUESTA SIN RESPUESTA YA QUE SON RESPECTO A ACTIVIDADES QUE PUEDEN O NO HACER


Valores nulos: id                                                           0
salud_actual                                                 0
nivel_de_movilidad                                           0
capacidad_cuidado_personal                                   0
capacidad_actividades_cotidianas                             0
dolor_malestar_general                                       0
ansiedad_depresion                                           0
af_intensa_veces_por_semana                                  0
af_intensa_min_totales_por_semana                        22842
af_intensa_min_totales_por_semana_nsnc                   29186
af_moderada_veces_por_semana                                 0
af_moderada_min_totales_por_semana                       18985
actividad_fisica_moderada_min_totales_por_semana_nsnc    29169
caminata_veces_por_semana                                    0
caminata_min_totales_por_semana                           8905
caminata_min_totales_por_semana_nsnc    

In [44]:
#evaluo los nulos y si las respuestas son excluyente, es decir que si hago intensa no hago moderada o caminata etc,,,
columnas_actFis = ['af_intensa_veces_por_semana', 'af_intensa_min_totales_por_semana', 'af_intensa_min_totales_por_semana_nsnc', 'af_moderada_veces_por_semana', 'af_moderada_min_totales_por_semana', 'actividad_fisica_moderada_min_totales_por_semana_nsnc', 'caminata_veces_por_semana']
salud_y_af[columnas_actFis].head(5)

# se aprecia que no son excluyentes las respuestas y que los nulos corresponden a otras respuestas respondida


,af_intensa_veces_por_semana,af_intensa_min_totales_por_semana,af_intensa_min_totales_por_semana_nsnc,af_moderada_veces_por_semana,af_moderada_min_totales_por_semana,actividad_fisica_moderada_min_totales_por_semana_nsnc,caminata_veces_por_semana
0,4,500.0,NaN,4,120.0,NaN,7
1,3,210.0,NaN,1,30.0,NaN,7
2,0,NaN,NaN,3,180.0,NaN,7
3,0,NaN,NaN,0,NaN,NaN,0
4,0,NaN,NaN,2,180.0,NaN,7


In [45]:
#TIPOS DE VARIABLES

#vemos que hay algunas columnas como flotante, para pasarlas a enteros ya que de lo contrario es complicado hacer análisis
#primero debemos elimira los nan en esas columnas


print(salud_y_af.dtypes)


columnas_actFis = ['af_intensa_min_totales_por_semana', 
                   'af_intensa_min_totales_por_semana_nsnc',
                   'af_moderada_min_totales_por_semana',
                   'actividad_fisica_moderada_min_totales_por_semana_nsnc' ,
                   'af_moderada_min_totales_por_semana',
                   'actividad_fisica_moderada_min_totales_por_semana_nsnc',
                   'caminata_min_totales_por_semana',
                   'caminata_min_totales_por_semana_nsnc',                     
                   'tiempo_sentado_min_por_dia',                              
                   'tiempo_sentado_min_por_dia_nsnc',                          
                   'razon_ninguna_actividad_la_ultima_semana',                
                   'razon_poca_actividad_la_ultima_semana',               
                   'af_fue_actividad_domestica',                            
                   'af_fue_actividad_laboral',                               
                   'af_fue_traslado',                                         
                   'af_fue_deporte',                                          
                   'nivel_actividad_fisica']



todas_nan_en_fila = salud_y_af[columnas_actFis].isna().all(axis=1)
fila_nan = salud_y_af[todas_nan_en_fila]
print(f'\n las que tienen todas nan son: {fila_nan}\n')

#como en ninguna fila todos los valores son nan (de las columnas seleccionadas) reemplazo nan con 0

salud_y_af[columnas_actFis]= salud_y_af[columnas_actFis].fillna(0)  # Reemplazar NaN con 0

salud_y_af[columnas_actFis] = salud_y_af[columnas_actFis].astype(int)
print(salud_y_af.dtypes)



id                                                         int64
salud_actual                                               int64
nivel_de_movilidad                                         int64
capacidad_cuidado_personal                                 int64
capacidad_actividades_cotidianas                           int64
dolor_malestar_general                                     int64
ansiedad_depresion                                         int64
af_intensa_veces_por_semana                                int64
af_intensa_min_totales_por_semana                        float64
af_intensa_min_totales_por_semana_nsnc                   float64
af_moderada_veces_por_semana                               int64
af_moderada_min_totales_por_semana                       float64
actividad_fisica_moderada_min_totales_por_semana_nsnc    float64
caminata_veces_por_semana                                  int64
caminata_min_totales_por_semana                          float64
caminata_min_totales_por_

In [46]:
print(f"Valores nulos: {salud_y_af.isnull().sum()}\n") 

Valores nulos: id                                                       0
salud_actual                                             0
nivel_de_movilidad                                       0
capacidad_cuidado_personal                               0
capacidad_actividades_cotidianas                         0
dolor_malestar_general                                   0
ansiedad_depresion                                       0
af_intensa_veces_por_semana                              0
af_intensa_min_totales_por_semana                        0
af_intensa_min_totales_por_semana_nsnc                   0
af_moderada_veces_por_semana                             0
af_moderada_min_totales_por_semana                       0
actividad_fisica_moderada_min_totales_por_semana_nsnc    0
caminata_veces_por_semana                                0
caminata_min_totales_por_semana                          0
caminata_min_totales_por_semana_nsnc                     0
tiempo_sentado_min_por_dia               

In [47]:
#elimino columnas sobre movilidad y salud actual ya que la forma de evaluar la salud actual es a partir de las enfermedades por lo que no serían causas
#sino consecuencias

salud_y_af = salud_y_af.drop(columns=["salud_actual",  "nivel_de_movilidad", "capacidad_cuidado_personal",
    "capacidad_actividades_cotidianas", "dolor_malestar_general","razon_ninguna_actividad_la_ultima_semana",
    "razon_poca_actividad_la_ultima_semana","barreras_actividad_fisica"])

In [48]:
salud_y_af.columns

Index(['id', 'ansiedad_depresion', 'af_intensa_veces_por_semana',
       'af_intensa_min_totales_por_semana',
       'af_intensa_min_totales_por_semana_nsnc',
       'af_moderada_veces_por_semana', 'af_moderada_min_totales_por_semana',
       'actividad_fisica_moderada_min_totales_por_semana_nsnc',
       'caminata_veces_por_semana', 'caminata_min_totales_por_semana',
       'caminata_min_totales_por_semana_nsnc', 'tiempo_sentado_min_por_dia',
       'tiempo_sentado_min_por_dia_nsnc', 'af_fue_actividad_domestica',
       'af_fue_actividad_laboral', 'af_fue_traslado', 'af_fue_deporte',
       'nivel_actividad_fisica'],
      dtype='object')

In [49]:
#encoder y escaler

df = salud_y_af

columnas_codificar = ['ansiedad_depresion', 'nivel_actividad_fisica']
columnas_escalar = ['af_intensa_veces_por_semana', 'af_intensa_min_totales_por_semana', 'af_intensa_min_totales_por_semana_nsnc', 'af_moderada_veces_por_semana', 'af_moderada_min_totales_por_semana', 'actividad_fisica_moderada_min_totales_por_semana_nsnc',
       'caminata_veces_por_semana', 'caminata_min_totales_por_semana', 'caminata_min_totales_por_semana_nsnc', 'tiempo_sentado_min_por_dia', 'tiempo_sentado_min_por_dia_nsnc' ]

columnas_codificadas = encoder.fit_transform(df[columnas_codificar])
df_codif = pd.DataFrame(columnas_codificadas, columns=encoder.get_feature_names_out(columnas_codificar))
df_codif = df_codif.astype(int)  # Convertir a int

#escalar 
df[columnas_escalar] = scaler.fit_transform(df[columnas_escalar])
                

#uno todo

df = df.drop(columnas_codificar, axis=1)
df = pd.concat([df, df_codif], axis=1)

salud_y_af = df

## 7) TABAQUISMO

In [50]:
tabaquismo.head(5)

,id,bita01,bita02,bita02_99,bita03,bita04,bita04_01,bita04_02,bita05,bita06_a,...,bita10_03,bita10_04,bita10_05,bita10_06,bita11,bita12,bita13,bita14,bita15,bita16
0,1128639,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2,98,2,1,2,2,1,NaN,2
1,1709939,1,20.0,NaN,2.0,3.0,NaN,NaN,NaN,NaN,...,2,2,2,2,1,2,2,2,NaN,1
2,6874130,1,18.0,NaN,1.0,3.0,NaN,NaN,NaN,NaN,...,98,2,2,1,1,2,2,1,NaN,2
3,10319375,1,14.0,NaN,1.0,1.0,2.0,1.0,1.0,1.0,...,2,2,1,2,2,2,2,1,2.0,1
4,11140857,1,13.0,NaN,2.0,3.0,NaN,NaN,NaN,NaN,...,2,2,2,2,1,2,2,1,NaN,99


In [51]:
# RENOMBRO COLUMNAS


tabaquismo.rename(columns = {
 
    "bita01" : "alguna_vez_fumo",
    "bita02" : "edad_fumo_primera_vez",
    "bita02_99" : "No_sabe_edad_fumo_primera_vez",
    "bita04" : "actualmente_fuma_cigarrillos",
 #   "bita10_01" : "ultimos_30dias_fumaron_casa",
 #   "bita10_02" : "ultimos_30dias_fumaron_trabajo",
  
    }, inplace = True)



In [52]:
#ELIMINO COLUMNAS INNECESARIAS

#bita03 si ha fumado 100 cigarrillos, bita04 sobre si fuma cigarrillo armado o de paquete y bita05 marca
#sobre tipos de cigarrillo que fuma, sobre publicidad para fumar o dejar cigarrillo, etc.. cuestiones no relevantes para el análisis actual

tabaquismo = tabaquismo.drop(columns =  ['bita03','bita04_01','bita04_02','bita05','bita06_a','bita06_b','bita06_b_99','bita06_c','bita06_d','bita07','bita07_99',
                           'bita08','bita09_01','bita09_02','bita09_03','bita09_04','bita09_05','bita09_06','bita10_01','bita10_02','bita10_03','bita10_04','bita10_05','bita10_06',
                           'bita11', 'bita12','bita13', 'bita14','bita15', 'bita16'])
                            
#'ta_paquete_y_armado','ta_dejar_fumar','ta_otros_productos', 'ta_perc_publicidad', 'ta_percepcion_riesgo', 'imagenes_tabaco'




In [53]:
#DICCIONARIO SI NECESITO


#actualmente_fuma_cigarrillos_dict = {
#    1: 'todos los dias',
#    2: 'algunos dias',
#    3: 'no fuma'

#}

#tabaquismo['actualmente_fuma_cigarrillos'] = tabaquismo['actualmente_fuma_cigarrillos'].map(actualmente_fuma_cigarrillos_dict)



#consumo_tabaco_100_dict = {
#    1: 'fumador actual',
#    2: 'ex fumador',
#    3: 'no fumador'

#}

#tabaquismo['consumo_tabaco_100_dict'] = tabaquismo['consumo_tabaco_100_dict'].map(consumo_tabaco_100_dict)



In [54]:
# VALORES NULOS

print(tabaquismo.shape)
print(tabaquismo.isnull().sum())

(29224, 5)
id                                   0
alguna_vez_fumo                      0
edad_fumo_primera_vez            14170
No_sabe_edad_fumo_primera_vez    29096
actualmente_fuma_cigarrillos     14042
dtype: int64


In [55]:
# reviso que si los que no respondieron edad fumo primera vez, son los que respondieron no fumo en la primer


print(f'Personas que si fumaron:{(tabaquismo["alguna_vez_fumo"]==1).sum()}')
print(f'Personas que no fumaron:{(tabaquismo["alguna_vez_fumo"]==2).sum()}')

Personas que si fumaron:15182
Personas que no fumaron:14042


### Análisis de nulos
Hay 15182 que si han fumado y  15054 que si respondieron "la edad fumo primerz vez" , quedan 128 que no
saben la edad fumo primerz vez
los que nunca fumaron son 14042 + 128 que no saben edad da 14170 que son los nulos de edad fumo primera vez
Luego 29224 (total) - 128 da 29096 que son el total de "no sabe edad fumo primera vez"

Este analisis explica los nulos en edad fumo primerz vez y no sabe edad fumo primera vez

Los nulos de actualmente fuma cigarrillos corresponden a las 14042 personas que nunca fumaron por lo que tampoco fuman actualmente


In [56]:
#tipos de variables

tabaquismo.dtypes

columnas_tabaq=['edad_fumo_primera_vez', 'No_sabe_edad_fumo_primera_vez', 'actualmente_fuma_cigarrillos']

tabaquismo[columnas_tabaq]= tabaquismo[columnas_tabaq].fillna(0) #se completan con 0 los nan ya que esa opcion no esta en las posib

tabaquismo[columnas_tabaq]= tabaquismo[columnas_tabaq].astype(int)   #convierto flotante a enteros para simplificar  análisis

tabaquismo.dtypes 

id                               int64
alguna_vez_fumo                  int64
edad_fumo_primera_vez            int32
No_sabe_edad_fumo_primera_vez    int32
actualmente_fuma_cigarrillos     int32
dtype: object

In [57]:
#encoder y escaler

df = tabaquismo

#columnas_codificar = ['ansiedad_depresion', 'nivel_actividad_fisica']
columnas_escalar = ['edad_fumo_primera_vez' ]

#columnas_codificadas = encoder.fit_transform(df[columnas_codificar])
#df_codif = pd.DataFrame(columnas_codificadas, columns=encoder.get_feature_names_out(columnas_codificar))
#df_codif = df_codif.astype(int)  # Convertir a int

#uno todo

#df = df.drop(columnas_codificar, axis=1)
#df = pd.concat([df, df_codif], axis=1)


#escalar 
df[columnas_escalar] = scaler.fit_transform(df[columnas_escalar])
                

tabaquismo = df

## 8) HIPERTENSIÓN

In [58]:
hipertension.head(5)

,id,biha01,biha02,biha03,biha04,biha05_01,biha05_02,biha06,biha06_99,biha07,...,biha09,biha10,biha11,biha11_99,biha12,biha13,biha14,biha15,control_hipertension,prevalencia_hipertension
0,1128639,1,1.0,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0
1,1709939,1,1.0,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0
2,6874130,1,1.0,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0
3,10319375,1,2.0,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0
4,11140857,1,1.0,3,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0


In [59]:
hipertension.rename(columns={
#    'biha01':'le_han_tomado_presion',
#    'biha02':'ultima_medicion_presion',
    'biha03':'veces_presion_alta',
#    'biha04':'tratamiento_presion_ultimasSemanas',
#    'biha05_01':'ejercicio_dieta',
#    'biha05_02':'medicamentos_presion',
#    'biha06':'pastillas_por_dia_presion',
#    'biha06_99':'pastillas_ns_nc',
#    'biha07':'olvida_pastilla',
#    'biha08':'deja_pastilla_si_esta_bien',
#    'biha09':'deja_pastilla_si_esta_mal',
#    'biha10':'pastillas_ultima_semana',
#    'biha11':'cuantas_NO_tomo',
#    'biha11_99':'cuantas_NO_tomo_ns_nc',
#    'biha12':'asistir_control_presion',
#    'biha13':'lugar_asistir_control_presion',
#    'biha14':'atiende_mismo_profesional',
#    'biha15':'medico_conoce_historia_clinica'
        }, inplace=True)

In [60]:
# ELIMINO COLUMNAS INNECESARIAS
#tienen que ver con el tipo de tratamiento de hipertension, cuando se ha controlado, las veces que se control, etc.

hipertension = hipertension.drop(columns =  ['biha01','biha02','biha04','biha05_01','biha05_02','biha06','biha06_99','biha07',
                           'biha08','biha09','biha10','biha11','biha11_99', 'biha12','biha13', 'biha14','biha15','control_hipertension','prevalencia_hipertension' ])
                            

In [61]:
# diccionario si necesito

#veces_presion_alta_dict = {
#    1: 'solo 1 vez',
#    2: 'mas de 1 vez',
#    3: 'ninguna',
#    99: 'NsNc'
#}

#hipertension['veces_presion_alta'] = hipertension['veces_presion_alta'].map(veces_presion_alta_dict)

In [62]:
#nulos

hipertension.isnull().sum()

id                    0
veces_presion_alta    0
dtype: int64

In [63]:
#tipos de variables

hipertension.dtypes

id                    int64
veces_presion_alta    int64
dtype: object

In [64]:
hipertension['veces_presion_alta'].unique()

array([ 3,  2,  1, 99], dtype=int64)

In [65]:
#unificamos los valores =1 que son equivalentes a presion alta solo 1 vez, con los valores 2 que son equivalentes a presión alta mas de 1 vez
# esta unificación simplifica la salida del modelo (para clasificar en si tiene o no)
#este punto debería llevar mas atención y probablemente consejo de un profesional en el área antes de realizarlo

hipertension['veces_presion_alta'] = hipertension['veces_presion_alta'].replace(2, 1)  #reemplazo los 2 por valor 1
hipertension['veces_presion_alta'] = hipertension['veces_presion_alta'].replace(3, 2 )  #reemplazo los 2 por valor 1

print(hipertension['veces_presion_alta'].unique())


[ 2  1 99]


In [66]:
#encoder y escaler

df = hipertension

columnas_codificar = ['veces_presion_alta']
#columnas_escalar = ['edad_fumo_primera_vez' ]

columnas_codificadas = encoder.fit_transform(df[columnas_codificar])
df_codif = pd.DataFrame(columnas_codificadas, columns=encoder.get_feature_names_out(columnas_codificar))
df_codif = df_codif.astype(int)  # Convertir a int

#uno todo

df = df.drop(columnas_codificar, axis=1)
df = pd.concat([df, df_codif], axis=1)


#escalar 
#df[columnas_escalar] = scaler.fit_transform(df[columnas_escalar])
                

hipertension = df

hipertension.head(10)

,id,veces_presion_alta_2,veces_presion_alta_99
0,1128639,1,0
1,1709939,1,0
2,6874130,1,0
3,10319375,1,0
4,11140857,1,0
5,12665714,1,0
6,13089112,0,0
7,19254804,1,0
8,19959160,1,0
9,21544403,0,0


In [67]:
#como veces_presion alta esta indicando los valores 2 que son NUNCA PRESION, vamos a invertir la variable

hipertension['veces_presion_alta_2'] = hipertension['veces_presion_alta_2'].replace(1, 2)  #reemplazo los 1 por valor 2
hipertension['veces_presion_alta_2'] = hipertension['veces_presion_alta_2'].replace(0, 1 )  #reemplazo los 0 por valor 1
hipertension['veces_presion_alta_2'] = hipertension['veces_presion_alta_2'].replace(2, 0)  #reemplazo los 2 por valor 0

In [68]:
hipertension.head(5)

,id,veces_presion_alta_2,veces_presion_alta_99
0,1128639,0,0
1,1709939,0,0
2,6874130,0,0
3,10319375,0,0
4,11140857,0,0


## 9) PESO CORPORAL

In [69]:
pesocorporal.head(5)

,id,bipc01,bipc02,bipc03,bipc04,bipc04_99,bipc05,bipc05_99,imc
0,1128639,1,1.0,NaN,104.0,NaN,181.0,NaN,31.75
1,1709939,2,NaN,1.0,65.0,NaN,160.0,NaN,25.39
2,6874130,2,NaN,2.0,46.0,NaN,160.0,NaN,17.97
3,10319375,2,NaN,2.0,55.0,NaN,155.0,NaN,22.89
4,11140857,2,NaN,1.0,68.0,NaN,178.0,NaN,21.46


In [70]:
# ELIMINO COLUMNAS INNECESARIAS

pesocorporal = pesocorporal.drop(columns =  ['bipc02','bipc03','bipc04_99','bipc05_99', 'imc'])
                            

In [71]:
#renombro columnas

pesocorporal.rename(columns={
    'bipc01':'profesional_indica_sobrepeso',
    'bipc04':'peso_kg',
#    'bipc04_99':'no_sabe_peso',
    'bipc05':'altura_cm',
#    'bipc05_99':'no_sabe_altura'
    
        }, inplace=True)

pesocorporal.head(5)

,id,profesional_indica_sobrepeso,peso_kg,altura_cm
0,1128639,1,104.0,181.0
1,1709939,2,65.0,160.0
2,6874130,2,46.0,160.0
3,10319375,2,55.0,155.0
4,11140857,2,68.0,178.0


In [72]:
#NULOS


pesocorporal.isnull().sum()



id                                 0
profesional_indica_sobrepeso       0
peso_kg                         1011
altura_cm                       1676
dtype: int64

In [73]:
#tipos de datos

pesocorporal.dtypes

id                                int64
profesional_indica_sobrepeso      int64
peso_kg                         float64
altura_cm                       float64
dtype: object


### Tratamiento de variables y columnas

En este caso no se convierten a enteros ya que la naturaleza de las medidas sugiere que puedan tener decimales. 

    

In [74]:
#encoder y escaler

df = pesocorporal

#columnas_codificar = ['']
columnas_escalar = ['peso_kg' ,'altura_cm']

#columnas_codificadas = encoder.fit_transform(df[columnas_codificar])
#df_codif = pd.DataFrame(columnas_codificadas, columns=encoder.get_feature_names_out(columnas_codificar))
#df_codif = df_codif.astype(int)  # Convertir a int

#uno todo

#df = df.drop(columnas_codificar, axis=1)
#df = pd.concat([df, df_codif], axis=1)


#escalar 
df[columnas_escalar] = scaler.fit_transform(df[columnas_escalar])
                

pesocorporal = df

## 10) ALIMENTACIÓN

In [75]:
alimentacion.head(5)

,id,bial01,bial02,bial03,bial03_99,bial04,bial04_99,bial05,bial05_99,bial06,bial06_99,bial07,bial08,bial09,bial10,promedio_fv_diario,consumo_fv,barreras_fyv
0,1128639,2,3,7.0,NaN,3.0,NaN,7.0,NaN,1.0,NaN,8,2,NaN,2,4.00,2,1
1,1709939,2,2,7.0,NaN,5.0,NaN,5.0,NaN,2.0,NaN,1,2,NaN,1,6.43,1,4
2,6874130,1,1,5.0,NaN,2.0,NaN,5.0,NaN,2.0,NaN,2,2,NaN,2,2.86,2,1
3,10319375,99,1,3.0,NaN,1.0,NaN,7.0,NaN,1.0,NaN,9,2,NaN,99,1.43,2,1
4,11140857,1,3,7.0,NaN,1.0,NaN,3.0,NaN,1.0,NaN,2,2,NaN,1,1.43,2,1


In [76]:
#renombro columnas

alimentacion.columns

alimentacion.rename(columns= {
    'bial08':'considera_su_alimentacion',
    'consumo_fv':'consumo_5porc_fv'
    
}, inplace=True)

In [77]:
#elimino columnas


alimentacion = alimentacion.drop(columns = ['bial01', 'bial02', 'bial03', 'bial03_99', 'bial04', 'bial04_99',
       'bial05', 'bial05_99', 'bial06', 'bial06_99', 'bial07','bial09', 'bial10', 'barreras_fyv'])

print(alimentacion.head(5))

         id  considera_su_alimentacion  promedio_fv_diario  consumo_5porc_fv
0   1128639                          2                4.00                 2
1   1709939                          2                6.43                 1
2   6874130                          2                2.86                 2
3  10319375                          2                1.43                 2
4  11140857                          2                1.43                 2


In [78]:
#DICCIONARIO 

#considera_su_alimentacion_dict = {
    
#    1: 'muy saludable',
#    2: 'bastante saludable',
#    3:'poco saludable',
#    4: 'nada saludable',
#    99: 'nsnc'
    
#}

#alimentacion['considera_su_alimentacion']= alimentacion['considera_su_alimentacion'].map('considera_su_alimentacion_dict')


#consumo_5porc_fv_dict = {
    
#    1: 'si',
#    2: 'no',
#    99: 'nsnc'
    
#}

#alimentacion['consumo_5porc_fv']= alimentacion['consumo_5porc_fv'].map('consumo_5porc_fv_dict')


In [79]:
#nulos

alimentacion.isnull().sum()

id                             0
considera_su_alimentacion      0
promedio_fv_diario           753
consumo_5porc_fv               0
dtype: int64

In [80]:
print((alimentacion['consumo_5porc_fv']==99).sum())




753


La cantidad de nulos en promedio corresponde a personas que no saben cuantas frutas o verdus comen

In [81]:
#encoder y escaler

df = alimentacion

columnas_codificar = ['considera_su_alimentacion','consumo_5porc_fv']
columnas_escalar = ['promedio_fv_diario']

columnas_codificadas = encoder.fit_transform(df[columnas_codificar])
df_codif = pd.DataFrame(columnas_codificadas, columns=encoder.get_feature_names_out(columnas_codificar))
df_codif = df_codif.astype(int)  # Convertir a int

#uno todo

df = df.drop(columnas_codificar, axis=1)
df = pd.concat([df, df_codif], axis=1)


#escalar 
df[columnas_escalar] = scaler.fit_transform(df[columnas_escalar])
                

alimentacion = df

## 11) COLESTEROL

In [82]:
colesterol.head(5)

,id,bico01,bico02,bico03,bico04,bico05_01,bico05_02,control_colesterol,prevalencia_colesterol
0,1128639,1,2.0,2,NaN,NaN,NaN,NaN,2.0
1,1709939,1,2.0,1,2.0,NaN,NaN,1.0,1.0
2,6874130,1,2.0,2,NaN,NaN,NaN,NaN,2.0
3,10319375,1,2.0,1,1.0,2.0,1.0,1.0,1.0
4,11140857,1,1.0,2,NaN,NaN,NaN,NaN,2.0


In [83]:
#renombro columnas

colesterol.rename( columns={
    'bico01': 'medido_colesterol',
    'bico02': 'fecha_med_colesterol',
    'bico03': 'colesterol_alto',
    'bico04': 'tratamiento_medico_col',
    'bico05_01': 'trat_habitos_col',
    'bico05_02': 'trat_farmac_col',
    'control_colesterol': 'autocontrol_col'
}, inplace= True)

colesterol.columns

Index(['id', 'medido_colesterol', 'fecha_med_colesterol', 'colesterol_alto',
       'tratamiento_medico_col', 'trat_habitos_col', 'trat_farmac_col',
       'autocontrol_col', 'prevalencia_colesterol'],
      dtype='object')

In [84]:
#diccionarios


#medido_colesterol_dict = { 
#    1: 'si',
#    2: 'no',
#    99: 'nsnc'   
#}

#colesterol['medido_colesterol']= colesterol['medido_colesterol'].map('medido_colesterol_dict')




#colesterol_alto_dict = { 
#    1: 'si',
#    2: 'no',
#    99: 'nsnc'   
#}

#colesterol['colesterol_alto']= colesterol['colesterol_alto'].map('colesterol_alto_dict')


#prevalencia_colesterol_dict = { 
#    1: 'si',
#    2: 'no',
#    99: 'nsnc'   
#}

#colesterol['prevalencia_colesterol']= colesterol['prevalencia_colesterol'].map('prevalencia_colesterol')


In [85]:
#elimino columnas

colesterol= colesterol.drop(columns = [ 'medido_colesterol','fecha_med_colesterol', 'tratamiento_medico_col', 'trat_habitos_col', 'trat_farmac_col',
       'autocontrol_col', 'prevalencia_colesterol' ])



In [86]:
colesterol.dtypes

id                 int64
colesterol_alto    int64
dtype: object

In [87]:
colesterol.isnull().sum()

id                 0
colesterol_alto    0
dtype: int64

In [88]:
#print((colesterol['medido_colesterol']==2).sum())
#print((colesterol['medido_colesterol']==99).sum())

print((colesterol['colesterol_alto']==99).sum())
print((colesterol['colesterol_alto']==99).sum())


419
419


Se observa que los nulos de prevalencia colesterol corresponden a los que 
NO se han medido el colesterol nunca + los que no saben si se han medido el colesterol

In [89]:
colesterol.head(5)

,id,colesterol_alto
0,1128639,2
1,1709939,1
2,6874130,2
3,10319375,1
4,11140857,2


In [90]:
#encoder y escaler

df = colesterol

columnas_codificar = ['colesterol_alto']
#columnas_escalar = ['promedio_fv_diario']

columnas_codificadas = encoder.fit_transform(df[columnas_codificar])
df_codif = pd.DataFrame(columnas_codificadas, columns=encoder.get_feature_names_out(columnas_codificar))
df_codif = df_codif.astype(int)  # Convertir a int

#uno todo

df = df.drop(columnas_codificar, axis=1)
df = pd.concat([df, df_codif], axis=1)


#escalar 
#df[columnas_escalar] = scaler.fit_transform(df[columnas_escalar])
                

colesterol = df

In [91]:
colesterol.head(5)

,id,colesterol_alto_2,colesterol_alto_99
0,1128639,1,0
1,1709939,0,0
2,6874130,1,0
3,10319375,0,0
4,11140857,1,0


In [92]:
#colesterol_alto 2 me indica lso que no tienen colesterol asique lo invierto

colesterol['colesterol_alto_2'] = colesterol['colesterol_alto_2'].replace(1, 2)  #reemplazo los 1 por valor 2
colesterol['colesterol_alto_2'] = colesterol['colesterol_alto_2'].replace(0, 1 )  #reemplazo los 0 por valor 1
colesterol['colesterol_alto_2'] = colesterol['colesterol_alto_2'].replace(2, 0)  #reemplazo los 2 por valor 0

In [93]:
colesterol.head(5)

,id,colesterol_alto_2,colesterol_alto_99
0,1128639,0,0
1,1709939,1,0
2,6874130,0,0
3,10319375,1,0
4,11140857,0,0


## 12) CONSUMO ALCOHOL

In [94]:
consumo_alcohol.head(5)

,id,bica01,bica02,bica03_01,bica03_02,bica03_99,bica04_01_b,bica04_01_c,bica04_02_b,bica04_02_c,...,bica05_01_b,bica05_01_c,bica05_02_b,bica05_02_c,bica05_03_b,bica05_03_c,bica05_04,bica06,bica07,consumo_regular_riesgo
0,1128639,1,1.0,1.0,NaN,NaN,1.0,NaN,NaN,98.0,...,3.0,NaN,1.0,NaN,NaN,98.0,2.0,2.0,2,2.0
1,1709939,1,1.0,NaN,1.0,NaN,NaN,98.0,1.0,NaN,...,NaN,98.0,NaN,98.0,NaN,98.0,2.0,2.0,2,2.0
2,6874130,1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2.0
3,10319375,1,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0
4,11140857,1,1.0,1.0,NaN,NaN,1.5,NaN,NaN,98.0,...,NaN,98.0,NaN,98.0,NaN,98.0,2.0,2.0,99,2.0


In [95]:
consumo_alcohol.columns


Index(['id', 'bica01', 'bica02', 'bica03_01', 'bica03_02', 'bica03_99',
       'bica04_01_b', 'bica04_01_c', 'bica04_02_b', 'bica04_02_c',
       'bica04_03_b', 'bica04_03_c', 'bica04_04', 'bica05_01_b', 'bica05_01_c',
       'bica05_02_b', 'bica05_02_c', 'bica05_03_b', 'bica05_03_c', 'bica05_04',
       'bica06', 'bica07', 'consumo_regular_riesgo'],
      dtype='object')

In [96]:
#rename columns

consumo_alcohol.rename(columns ={
    
    'bica01':'consumido_alcohol'   
    
}, inplace=True)


In [97]:
consumo_alcohol =consumo_alcohol.drop(columns=['bica02', 'bica03_01', 'bica03_02', 'bica03_99',
       'bica04_01_b',
        'bica04_01_c', 'bica04_02_b', 'bica04_02_c',
       'bica04_03_b', 'bica04_03_c', 'bica04_04', 'bica05_01_b', 'bica05_01_c',
       'bica05_02_b', 'bica05_02_c', 'bica05_03_b', 'bica05_03_c', 'bica05_04',
       'bica06', 'bica07'])

In [98]:
consumo_alcohol.dtypes

id                          int64
consumido_alcohol           int64
consumo_regular_riesgo    float64
dtype: object

In [99]:
#cantidad de nulos


print(consumo_alcohol.isnull().sum())

print((consumo_alcohol['consumido_alcohol']==99).sum())

id                          0
consumido_alcohol           0
consumo_regular_riesgo    154
dtype: int64
36


In [100]:
#encoder y escaler

df = consumo_alcohol

columnas_codificar = ['consumido_alcohol']
#columnas_escalar = ['promedio_fv_diario']

columnas_codificadas = encoder.fit_transform(df[columnas_codificar])
df_codif = pd.DataFrame(columnas_codificadas, columns=encoder.get_feature_names_out(columnas_codificar))
df_codif = df_codif.astype(int)  # Convertir a int

#uno todo

df = df.drop(columnas_codificar, axis=1)
df = pd.concat([df, df_codif], axis=1)


#escalar 
#df[columnas_escalar] = scaler.fit_transform(df[columnas_escalar])
                

consumo_alcohol = df

## 13) DIABETES

In [101]:
diabetes.head(5)

,id,bidi01,bidi02,bidi03,bidi04_01,bidi04_02,bidi04_03,bidi05,bidi06_01,bidi06_02,bidi07,bidi08,bidi09,bidi10,bidi11,bidi12,bidi13,bidi14,control_diabetes
0,1128639,2,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,1709939,2,NaN,NaN,NaN,NaN,NaN,NaN,2,2,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,6874130,2,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,10319375,2,NaN,NaN,NaN,NaN,NaN,NaN,2,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,11140857,2,NaN,NaN,NaN,NaN,NaN,NaN,2,99,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [102]:
diabetes.columns

Index(['id', 'bidi01', 'bidi02', 'bidi03', 'bidi04_01', 'bidi04_02',
       'bidi04_03', 'bidi05', 'bidi06_01', 'bidi06_02', 'bidi07', 'bidi08',
       'bidi09', 'bidi10', 'bidi11', 'bidi12', 'bidi13', 'bidi14',
       'control_diabetes'],
      dtype='object')

In [103]:
#drop columnas que tienen que ver con tratamiento y como se realizó el control

diabetes= diabetes.drop(columns = ['bidi02','bidi03', 'bidi04_01', 'bidi04_02',
       'bidi04_03', 'bidi05', 'bidi07', 'bidi08',
       'bidi09', 'bidi10', 'bidi11', 'bidi12', 'bidi13', 'bidi14', 'control_diabetes'])

In [104]:
#rename

diabetes.rename(columns={
      'bidi01':'dijeron_diabetes',
#      'bidi02':'fue_embarazo',  
      'bidi06_01':'familiares_directos_diabetes',
      'bidi06_02':'familiares_no_directos_diabetes',
#      'bidi07': 'medicion_azucar'
    
}, inplace=True)

diabetes.columns

Index(['id', 'dijeron_diabetes', 'familiares_directos_diabetes',
       'familiares_no_directos_diabetes'],
      dtype='object')

In [105]:
# tipos de variables

diabetes.dtypes

id                                 int64
dijeron_diabetes                   int64
familiares_directos_diabetes       int64
familiares_no_directos_diabetes    int64
dtype: object

In [106]:
#nulos

diabetes.isnull().sum()

id                                 0
dijeron_diabetes                   0
familiares_directos_diabetes       0
familiares_no_directos_diabetes    0
dtype: int64

In [107]:
print((diabetes['dijeron_diabetes']==1).sum())
print((diabetes['dijeron_diabetes']==2).sum())
print((diabetes['dijeron_diabetes']==99).sum())

3924
25165
135


In [108]:
diabetes.shape

(29224, 4)

In [109]:
25165+135

25300

In [110]:
#encoder y escaler

df = diabetes

columnas_codificar = ['dijeron_diabetes', 'familiares_directos_diabetes','familiares_no_directos_diabetes']
#columnas_escalar = ['promedio_fv_diario']

columnas_codificadas = encoder.fit_transform(df[columnas_codificar])
df_codif = pd.DataFrame(columnas_codificadas, columns=encoder.get_feature_names_out(columnas_codificar))
df_codif = df_codif.astype(int)  # Convertir a int

#uno todo

df = df.drop(columnas_codificar, axis=1)
df = pd.concat([df, df_codif], axis=1)


#escalar 
#df[columnas_escalar] = scaler.fit_transform(df[columnas_escalar])
                

diabetes = df

In [111]:
diabetes.head(5)


,id,dijeron_diabetes_2,dijeron_diabetes_99,familiares_directos_diabetes_2,familiares_directos_diabetes_99,familiares_no_directos_diabetes_2,familiares_no_directos_diabetes_99
0,1128639,1,0,1,0,1,0
1,1709939,1,0,1,0,1,0
2,6874130,1,0,1,0,1,0
3,10319375,1,0,1,0,0,0
4,11140857,1,0,1,0,0,1


In [112]:
#colesterol_alto 2 me indica lso que no tienen colesterol asique lo invierto

diabetes['dijeron_diabetes_2'] = diabetes['dijeron_diabetes_2'].replace(1, 2)  #reemplazo los 1 por valor 2
diabetes['dijeron_diabetes_2'] = diabetes['dijeron_diabetes_2'].replace(0, 1 )  #reemplazo los 0 por valor 1
diabetes['dijeron_diabetes_2'] = diabetes['dijeron_diabetes_2'].replace(2, 0)  #reemplazo los 2 por valor 0

In [113]:
diabetes.head(5)

,id,dijeron_diabetes_2,dijeron_diabetes_99,familiares_directos_diabetes_2,familiares_directos_diabetes_99,familiares_no_directos_diabetes_2,familiares_no_directos_diabetes_99
0,1128639,0,0,1,0,1,0
1,1709939,0,0,1,0,1,0
2,6874130,0,0,1,0,1,0
3,10319375,0,0,1,0,0,0
4,11140857,0,0,1,0,0,1


## UNO LAS COLUMNAS Y GENERO CSV PARA TRABAJO

In [114]:
friesgo_2018_reduc = dominio_geografico.merge(caracteristicas_vivienda, on= 'id', how='outer' ).merge(caracteristicas_hogar, on= 'id', how='outer').merge(ingresos_hogar, on= 'id', how='outer').merge(caracteristicas_encuestado, on= 'id', how='outer')  

friesgo_2018_reduc = friesgo_2018_reduc.merge(salud_y_af, on= 'id', how='outer' ).merge(tabaquismo, on= 'id', how='outer' ).merge(hipertension, on= 'id', how='outer' ).merge(pesocorporal, on= 'id', how='outer' )

friesgo_2018_reduc = friesgo_2018_reduc.merge(alimentacion, on= 'id', how='outer' ).merge(colesterol, on= 'id', how='outer' ).merge(consumo_alcohol, on= 'id', how='outer' ).merge(diabetes, on= 'id', how='outer' )




friesgo_2018_reduc.head(5)



,id,region,tamanio_aglomerado_2,tamanio_aglomerado_3,tamanio_aglomerado_4,tipo_vivienda_2,tipo_vivienda_3,tipo_vivienda_4,tipo_vivienda_5,tipo_vivienda_6,...,colesterol_alto_99,consumo_regular_riesgo,consumido_alcohol_2,consumido_alcohol_99,dijeron_diabetes_2,dijeron_diabetes_99,familiares_directos_diabetes_2,familiares_directos_diabetes_99,familiares_no_directos_diabetes_2,familiares_no_directos_diabetes_99
0,1128639,1,0,0,0,0,1,0,0,0,...,0,2.0,0,0,0,0,1,0,1,0
1,1709939,1,0,0,0,0,0,0,0,0,...,0,2.0,0,0,0,0,1,0,1,0
2,6874130,1,0,0,0,0,1,0,0,0,...,0,2.0,0,0,0,0,1,0,1,0
3,10319375,1,0,0,0,0,1,0,0,0,...,0,2.0,0,0,0,0,1,0,0,0
4,11140857,1,0,0,0,0,1,0,0,0,...,0,2.0,0,0,0,0,1,0,0,1


In [115]:
friesgo_2018_reduc.columns

Index(['id', 'region', 'tamanio_aglomerado_2', 'tamanio_aglomerado_3',
       'tamanio_aglomerado_4', 'tipo_vivienda_2', 'tipo_vivienda_3',
       'tipo_vivienda_4', 'tipo_vivienda_5', 'tipo_vivienda_6',
       ...
       'colesterol_alto_99', 'consumo_regular_riesgo', 'consumido_alcohol_2',
       'consumido_alcohol_99', 'dijeron_diabetes_2', 'dijeron_diabetes_99',
       'familiares_directos_diabetes_2', 'familiares_directos_diabetes_99',
       'familiares_no_directos_diabetes_2',
       'familiares_no_directos_diabetes_99'],
      dtype='object', length=101)

In [116]:
friesgo_2018_reduc.shape

(29224, 101)

In [117]:

#analizo cuantas filas elimino si dan 99 (NsNc) en la salida de las variables que se evalúan: colesterol, diabetes e hipertension
print(f"veces_presion_alta = 99 (no sabe): {((friesgo_2018_reduc['veces_presion_alta_99'])==1).sum()}") 
print(f"Colesterol_alto = 99 (no sabe): {((friesgo_2018_reduc['colesterol_alto_99'])==1).sum()}")
print(f"Dijeron_ diabetes = 99 (no sabe): {((friesgo_2018_reduc['dijeron_diabetes_99'])==1).sum()}")

friesgo_2018_reduc = friesgo_2018_reduc[friesgo_2018_reduc['veces_presion_alta_99'] != 1]
friesgo_2018_reduc = friesgo_2018_reduc[friesgo_2018_reduc['colesterol_alto_99'] != 1]
friesgo_2018_reduc = friesgo_2018_reduc[friesgo_2018_reduc['dijeron_diabetes_99'] != 1]


print(f"veces_presion_alta = 99 (no sabe): {((friesgo_2018_reduc['veces_presion_alta_99'])==1).sum()}") 
print(f"Colesterol_alto = 99 (no sabe): {((friesgo_2018_reduc['colesterol_alto_99'])==1).sum()}")
print(f"Dijeron_ diabetes = 99 (no sabe): {((friesgo_2018_reduc['dijeron_diabetes_99'])==1).sum()}")

veces_presion_alta = 99 (no sabe): 212
Colesterol_alto = 99 (no sabe): 419
Dijeron_ diabetes = 99 (no sabe): 135
veces_presion_alta = 99 (no sabe): 0
Colesterol_alto = 99 (no sabe): 0
Dijeron_ diabetes = 99 (no sabe): 0


In [118]:
friesgo_2018_reducs= friesgo_2018_reduc.drop(columns = ['veces_presion_alta_99','colesterol_alto_99', 'dijeron_diabetes_99'])

In [119]:
#analizamos si quedan NAN en columnas

nan_counts = friesgo_2018_reduc.isna().sum()
print("Valores NaN por columna:")
print(nan_counts[nan_counts > 0])

Valores NaN por columna:
peso_kg                    957
altura_cm                 1606
promedio_fv_diario         703
consumo_regular_riesgo     150
dtype: int64


In [120]:
#analizamos cuantas filas son las que contienen NAN en total y si estan superpuestos los nan
#no se eliminaron filas antes porque debia estar todo el dataset unido y además
#lso valores que quedaban no pueden reemplazarse por un numero cualquiera ya que la variable
#correspondne a valores numpericos, no son codigos


columnas_con_nan = ['peso_kg', 'altura_cm', 'promedio_fv_diario', 'consumo_regular_riesgo']

filas_con_nan = friesgo_2018_reduc[columnas_con_nan].isna().any(axis=1)

# Contar el número total de filas que se eliminarían
filas_totales_nan = filas_con_nan.sum()

print(f"Número total de filas que se eliminarían: {filas_totales_nan}")

Número total de filas que se eliminarían: 2895


In [121]:
#eliminamos esas filas que en total representan un 10,38% del dataset total

friesgo_2018_reduc = friesgo_2018_reduc.dropna(subset=columnas_con_nan)

friesgo_2018_reduc.shape

(25637, 101)

In [122]:
#rename

friesgo_2018_reduc.rename(columns={
      'dijeron_diabetes_2':'dijeron_diabetes', 
      'colesterol_alto_2':'colesterol_alto',
      'veces_presion_alta_2':'veces_presion_alta'
    
}, inplace=True)

friesgo_2018_reduc.columns  

Index(['id', 'region', 'tamanio_aglomerado_2', 'tamanio_aglomerado_3',
       'tamanio_aglomerado_4', 'tipo_vivienda_2', 'tipo_vivienda_3',
       'tipo_vivienda_4', 'tipo_vivienda_5', 'tipo_vivienda_6',
       ...
       'colesterol_alto_99', 'consumo_regular_riesgo', 'consumido_alcohol_2',
       'consumido_alcohol_99', 'dijeron_diabetes', 'dijeron_diabetes_99',
       'familiares_directos_diabetes_2', 'familiares_directos_diabetes_99',
       'familiares_no_directos_diabetes_2',
       'familiares_no_directos_diabetes_99'],
      dtype='object', length=101)

In [123]:
#guardo df reducido en csv

friesgo_2018_reduc.to_csv('friesgo_2018_reduc.csv', sep = ',', encoding = 'utf-8', index=False)

## CORRECCIONES REALIZADAS EN EL DATA SET LUEGO DE LA PRIMER LIMPIEZA

1. En primer lugar una vez que se obtuvieron las salidas de la importancia de cada variable, se volvió a revisar el dataset de entrada y se eliminaron aquellas columnas cuyos valores no tenian significancia como causas de la variable, como por ejemplo si había asistido a consultas o si se había realizado un análisis recientemente. Estas columnas no dejan de ser importantes y pueden responder otras preguntas, pero no son el objetivo de este análisis.
2. La importancia de las variables se limitó a aquellas con una contribución al resultado del modelo mayores al 5%, esto dejaba aproximadamente las 3 o 4 variables más importantes en la determinación y permite comparar entre regiones de forma más dinámica.
3. Para simplificar la salida de la matriz de confusión se modificaron las opciones en las columnas:'veces_presion_alta': la salida  solo 1 vez se unificó con la salida + de 1 vez y se eliminan los valores nsnc; 'dijeron_diabetes':se eliminan los valores nsnc; 'colesterol_alto': se eliminan los valores nsnc

### Correcciones 2da vuelta
1. Se codifican variables que tenían 3 o más categorías aunque fuera numéricas
2. Se escalan valores 